In [1]:
pip install arxiv


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6078 sha256=72f2f7807c31ade68fe6506426562bc4525ca44906f28d3d91f9f4ce1b097b91
  Stored in directory: c:\users\vidha\appdata\local\pip\cache\wheels\83\63\2f\117884c3b19d46b64d3d61690333aa80c88dc14050e269c546
Successfully built sgmllib3k
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [6]:
pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11710 sha256=913c2c8075bae7a72c20231e82c6113293f385a779c8b50033e94bb69a4f6d3e
  Stored in directory: c:\users\vidha\appdata\local\pip\cache\wheels\07\93\05\72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [8]:
wiki.name

'wikipedia'

In [42]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://www.cancer.gov/about-cancer/causes-prevention/genetics")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018F6ABC6FA0>)

In [53]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Cancernotes.pdf")
docs = loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriever1=vectordb.as_retriever()
retriever1

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018F6ABB4310>)

In [54]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"cancer_prevention_search",
                      "Search for information about Cancer prevention. For any questions about Cancer prevention, you must use this tool!")

In [57]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool1=create_retriever_tool(retriever1,"cancer_prevention_pdf_search",
                      "Search for information about Cancer prevention. For any questions about Cancer prevention, you must use this tool!")

In [56]:
retriever_tool.name

'cancer_prevention_search'

In [58]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [59]:
tools=[retriever_tool,retriever_tool1,wiki,arxiv]

In [60]:
tools

[Tool(name='cancer_prevention_search', description='Search for information about Cancer prevention. For any questions about Cancer prevention, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000018F77BBBCA0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018F6ABC6FA0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000018F77BBBDC0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018F6ABC6FA0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 Tool(name='cancer_prevention_pdf_sea

In [62]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [63]:


from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [64]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [65]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_comm

In [66]:
agent_executor.invoke({"input":"Tell me about Cancer Prevention"})



> Entering new AgentExecutor chain...

Invoking: `cancer_prevention_search` with `{'query': 'Cancer Prevention'}`


As cancer cells divide, they acquire more DNA changes over time. Two cancer cells in the same tumor can have different DNA changes. In addition, every person with cancer has a unique combination of DNA changes in their cancer. 
For more information on the biological changes that make cells cancerous, see What is Cancer? Differences between Cancer Cells and Normal Cells.

Multiple kinds of genetic changes can lead to cancer. One genetic change, called a DNA mutation or genetic variant, is a change in the DNA code, like a typo in the sequence of DNA letters. 
Some variants affect just one DNA letter, called a nucleotide. A nucleotide may be missing, or it may be replaced by another nucleotide. These are called point mutations.
For example, around 5% of people with cancer have a point mutation in the KRAS gene that replaces the DNA letter G with A. This single letter chang

{'input': 'Tell me about Cancer Prevention',
 'output': "Cancer prevention involves understanding the genetic changes that can lead to cancer. Cancer cells acquire more DNA changes over time, and each person with cancer has a unique combination of DNA changes in their cancer cells. Genetic changes, such as DNA mutations or genetic variants, can alter the way cells grow and spread, leading to cancer.\n\nSome genetic changes, like point mutations, affect single DNA letters, while others involve rearrangements, deletions, or duplications of DNA segments. These changes can impact genes responsible for controlling cell growth, division, and self-destruction. For a healthy cell to become cancerous, multiple DNA changes usually need to occur.\n\nIt's important to note that people who have inherited cancer-related genetic changes may be more susceptible to developing cancer with fewer additional changes. Legal protections, such as the Genetic Information Nondiscrimination Act of 2008 (GINA) an